<a href="https://colab.research.google.com/github/Vaycold/statistics_seminar/blob/main/model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
os.environ['KAGGLE_USERNAME'] = 'kimgodbang'
os.environ['KAGGLE_KEY'] = '1c0e1716b23829d4381dcbced37ba49b'

In [4]:
!rm *.*
!kaggle competitions download -c sentiment-analysis-of-covid-19-related-tweets

  0% 0.00/254k [00:00<?, ?B/s]
100% 254k/254k [00:00<00:00, 38.2MB/s]
  0% 0.00/518k [00:00<?, ?B/s]
100% 518k/518k [00:00<00:00, 34.6MB/s]


In [5]:
df = pd.read_csv('training.csv')

In [6]:
df.shape

(5000, 3)

In [7]:
onehot = np.zeros((5000,11))
onehot.shape

(5000, 11)

In [8]:
def labelsplit(labels) :
    
    label = labels.replace(' ',',').split(',')
    for idx, lab in enumerate(label) :
        label[idx] = int(lab)
    return label

In [9]:
df['label']  = df['Labels'].apply(lambda x :  labelsplit(x))

In [10]:
df.head()

,ID,Tweet,Labels,label
0,1,NO JOKE I WILL HOP ON A PLANE RN! (Well after ...,0 10,"[0, 10]"
1,2,BanMediaHouse whose is responsible for spreadi...,6,[6]
2,3,Im waiting for someone to say to me that all t...,3 4,"[3, 4]"
3,4,He is a liar. Proven day night. Time again. Li...,6,[6]
4,5,"NEW: U.S. CoronaVirus death toll reaches 4,000...",8,[8]


In [11]:
onehot = np.zeros((5000,11))
for i in range(5000) : # 0~5000 row
    for j in df.label[i] : # j = value   
        onehot[i,j]  = 1

In [12]:
all_tweet = np.array(df.Tweet.values)
with open('./all_tweet.txt', 'w', encoding='utf-8') as f:
    for line in all_tweet : 
        if len(str(line).strip()) == 0 : continue
        try : 
            f.write(line + '\n')
        except : continue

In [13]:
!cat all_tweet.txt |head -n10

NO JOKE I WILL HOP ON A PLANE RN! (Well after COVID19 lol)
BanMediaHouse whose is responsible for spreading Fake and communal Stories in this pandemic Corona situation.
Im waiting for someone to say to me that all this corona thing is just an April fools joke
He is a liar. Proven day night. Time again. Lies when the truth will do. COVID19
NEW: U.S. CoronaVirus death toll reaches 4,000 after nearly 900 new deaths were reported today (BNO News) Covid19 CoronaVirusOutbreak
Coronavirus impact Govt extends I-T deadlines related to Sections 80C, 80D
42,000 people might have died in China from Covid19 but China was underreporting according to sources.
Dear Chinese! Kindly cook your bat thoroughly next time. Regards, COVID19 CoronaVirusUpdates CoronaVirus
This is how the govt of kenya is checking the temperatures of COVID19 and saying it is still low. Mtashangaa sana.
My mental health hasn't suffered at all under the coronavirus quarantine! Ha-ha, April Fools.


In [14]:
!pip install sentencepiece
import sentencepiece as spm

     |████████████████████████████████| 1.2MB 15.3MB/s 


In [15]:
spm.SentencePieceTrainer.train('--input=all_tweet.txt --model_prefix=m --vocab_size=8987')
sp =spm.SentencePieceProcessor()
sp.load('m.model')

True

In [16]:
train_df = df[:4000]
test_df  = df[4000:]
print(train_df.shape, test_df.shape)

(4000, 4) (1000, 4)


In [17]:
train_onehot = onehot[:4000]
test_onehot = onehot[4000:]

In [18]:
print(train_onehot.shape, test_onehot.shape)

(4000, 11) (1000, 11)


In [19]:
train_df['Tweet']

0       NO JOKE I WILL HOP ON A PLANE RN! (Well after ...
1       BanMediaHouse whose is responsible for spreadi...
2       Im waiting for someone to say to me that all t...
3       He is a liar. Proven day night. Time again. Li...
4       NEW: U.S. CoronaVirus death toll reaches 4,000...
                              ...                        
3995    You wouldnt have to worry about getting the co...
3996    i'm gonna go ahead and say it, i fucking hate ...
3997    It tells why kerala has exploding case of coro...
3998    ready for corona to call it a joke haha you al...
3999    is my car loan emi (7/4/2020) dedecated from m...
Name: Tweet, Length: 4000, dtype: object

In [20]:
train_df['bow'] = train_df['Tweet'].apply(lambda x : sp.encode_as_ids(str(x)))
test_df['bow'] = test_df['Tweet'].apply(lambda x : sp.encode_as_ids(str(x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [21]:
train_df.head()

,ID,Tweet,Labels,label,bow
0,1,NO JOKE I WILL HOP ON A PLANE RN! (Well after ...,0 10,"[0, 10]","[1535, 1582, 10, 1444, 6555, 1481, 77, 4838, 5..."
1,2,BanMediaHouse whose is responsible for spreadi...,6,[6],"[1921, 4543, 4546, 79, 506, 14, 2075, 148, 19,..."
2,3,Im waiting for someone to say to me that all t...,3 4,"[3, 4]","[76, 182, 8, 19, 245, 6, 82, 6, 37, 28, 49, 21..."
3,4,He is a liar. Proven day night. Time again. Li...,6,[6],"[360, 14, 9, 2977, 3, 4436, 4293, 102, 552, 3,..."
4,5,"NEW: U.S. CoronaVirus death toll reaches 4,000...",8,[8],"[1571, 42, 306, 3, 137, 3, 269, 201, 525, 854,..."


In [22]:
train_text = train_df['bow'].values
test_text = test_df['bow'].values

In [23]:
train_text.shape, test_text.shape

((4000,), (1000,))

In [24]:
train_text[:2]

array([list([1535, 1582, 10, 1444, 6555, 1481, 77, 4838, 5224, 46, 220, 4549, 193, 20, 17, 369, 160]),
       list([1921, 4543, 4546, 79, 506, 14, 2075, 148, 19, 443, 2883, 15, 2588, 157, 8420, 18, 21, 155, 24, 581, 3])],
      dtype=object)

In [25]:
train_bow_text = tf.keras.preprocessing.sequence.pad_sequences(train_text)
test_bow_text = tf.keras.preprocessing.sequence.pad_sequences(test_text)

In [26]:
train_bow_text[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0, 1535, 1582,   10, 1444, 6555,
       1481,   77, 4838, 5224,   46,  220, 4549,  193,   20,   17,  369,
        160], dtype=int32)

In [27]:
train_bow_text.shape

(4000, 45)

In [28]:
test_bow_text.shape

(1000, 44)

In [29]:
import collections 
word_counter = collections.Counter()

for text in train_text :
    word_counter.update(text)
for text in test_text :
    word_counter.update(text)


In [30]:
def n_cut(texts, n) :
    return np.array([[word for word in text if word_counter[word] >= n ] for text in texts])

In [31]:
import warnings
warnings.filterwarnings('ignore')

In [32]:
train_cut_text = n_cut(train_text,10)
test_cut_text = n_cut(test_text,10)

train_cut_text.shape, test_cut_text.shape

((4000,), (1000,))

In [33]:
train_cut_bow_text = tf.keras.preprocessing.sequence.pad_sequences(train_cut_text, maxlen=80)
test_cut_bow_text = tf.keras.preprocessing.sequence.pad_sequences(test_cut_text, maxlen=80)

In [34]:
train_cut_bow_text.shape, test_cut_bow_text.shape

((4000, 80), (1000, 80))

In [35]:
train_onehot.shape, test_onehot.shape

((4000, 11), (1000, 11))

In [36]:
# bow -> text
def bow2txt(bow) :
    return sp.decode_ids(bow)

In [37]:
bow2txt(test_text[2])

'On the Coronavirus, the dock media is doing Hindu-Muslim, you should make a genuine news on it.'

## MODELING

In [84]:
from keras.layers import Input, Dense, GRU, Embedding, SimpleRNN
from keras.models import Model
def seq2seq() :
    input_x_bow = Input(shape=(80,))
    embedding = Embedding(8987,160)
    x = embedding(input_x_bow)
    #l1 = SimpleRNN(100, return_sequences = True)(x)
    #l2 = SimpleRNN(100)(l1)
    #z = GRU(128)(x)
    #z = GRU(64)(l2)
    z = GRU(64)(x)
    #z = GRU()(z)
   
    #z = GRU(64)(z)


    y = Dense(11, activation = 'sigmoid')(z)

    model = Model(input_x_bow, y)
    model.compile(
        loss = 'binary_crossentropy',
        optimizer = 'adam',
        metrics = ['accuracy']
    )

    return model

In [85]:
model = seq2seq()
model.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 80)]              0         
_________________________________________________________________
embedding_11 (Embedding)     (None, 80, 160)           1437920   
_________________________________________________________________
gru_14 (GRU)                 (None, 64)                43392     
_________________________________________________________________
dense_8 (Dense)              (None, 11)                715       
Total params: 1,482,027
Trainable params: 1,482,027
Non-trainable params: 0
_________________________________________________________________


In [86]:
hist = model.fit(train_cut_bow_text, train_onehot,
                 validation_data = (test_cut_bow_text, test_onehot),
                 epochs = 20, 
                 verbose = 1)

Epoch 1/20
125/125 [==============================] - 7s 43ms/step - loss: 0.5410 - accuracy: 0.1086 - val_loss: 0.4168 - val_accuracy: 0.0970
Epoch 2/20
125/125 [==============================] - 5s 38ms/step - loss: 0.4147 - accuracy: 0.1182 - val_loss: 0.4093 - val_accuracy: 0.1670
Epoch 3/20
125/125 [==============================] - 5s 38ms/step - loss: 0.3917 - accuracy: 0.1757 - val_loss: 0.3986 - val_accuracy: 0.1760
Epoch 4/20
125/125 [==============================] - 5s 38ms/step - loss: 0.3597 - accuracy: 0.2220 - val_loss: 0.4002 - val_accuracy: 0.2140
Epoch 5/20
125/125 [==============================] - 5s 38ms/step - loss: 0.3407 - accuracy: 0.2539 - val_loss: 0.4028 - val_accuracy: 0.2410
Epoch 6/20
125/125 [==============================] - 5s 38ms/step - loss: 0.3163 - accuracy: 0.2957 - val_loss: 0.4052 - val_accuracy: 0.2790
Epoch 7/20
125/125 [==============================] - 5s 38ms/step - loss: 0.2963 - accuracy: 0.3485 - val_loss: 0.4160 - val_accuracy: 0.2730

In [96]:
model.predict(test_cut_bow_text[7:8])

array([[2.3848858e-01, 7.6638773e-04, 3.4643221e-03, 4.4506885e-02,
        2.6591459e-02, 2.8557388e-02, 9.8970068e-01, 9.2743207e-03,
        2.0192641e-04, 1.0155970e-03, 1.0814361e-02]], dtype=float32)

In [95]:
test_onehot[7]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])

In [69]:
pd.read_csv('validation.csv')

,ID,Tweet
0,5001,Forgot to a math test and I was failing but my...
1,5002,Corona effected came from Delhi in Bihar lakhi...
2,5003,Make CORONA END NA
3,5004,Imagine if the coronavirus pandemic was a big ...
4,5005,Howdy Ana. Where can I get married during the ...
...,...,...
2495,7496,Ease to re-search - Coronavirus started mainly...
2496,7497,If J N U is postering in favour of corona clus...
2497,7498,Gods word is powerful and I promise that if yo...
2498,7499,"My mom just made me drink boiled lemon, garlic..."
